In [1]:
from easydict import EasyDict as ED
import torch
# ^^^ pyforest auto-imports - don't write above this line
%load_ext autoreload
%autoreload 2

In [2]:
from cfg import TrainCfg, TrainCfg_ns, ModelCfg, ModelCfg_ns
from model import ECG_CRNN_CINC2020
from dataset import CINC2020
from trainer import CINC2020Trainer

from tqdm.auto import tqdm
from copy import deepcopy

from torch.nn.parallel import DistributedDataParallel as DDP, DataParallel as DP

In [3]:
ECG_CRNN_CINC2020.__DEBUG__ = False
CINC2020Trainer.__DEBUG__ = False
CINC2020.__DEBUG__ = False

In [4]:
TrainCfg_ns.db_dir = "/home/wenhao/Jupyter/wenhao/data/CinC2021/"
TrainCfg.db_dir = "/home/wenhao/Jupyter/wenhao/data/CinC2021/"

In [5]:
ds_train = CINC2020(TrainCfg_ns, training=True, lazy=False)
ds_val = CINC2020(TrainCfg_ns, training=False, lazy=False)


start performing train test split...





start performing train test split...



## resnet_nature_comm_bottle_neck_se, 1-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle

In [6]:
train_config = deepcopy(TrainCfg_ns)
train_config.cnn_name = "resnet_nature_comm_bottle_neck_se"
train_config.rnn_name = "none"
train_config.attn_name = "none"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

model_config = deepcopy(ModelCfg_ns)

model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name
model_config.clf = ED()
model_config.clf.out_channels = [
  # not including the last linear layer, whose out channels equals n_classes
]
model_config.clf.bias = True
model_config.clf.dropouts = 0.0
model_config.clf.activation = "mish"  # for a single layer `SeqLin`, activation is ignored

In [7]:
model = ECG_CRNN_CINC2020(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [8]:
model.module_size_

'33.4M'

In [9]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

DataParallel(
  (module): ECG_CRNN_CINC2020(
    (cnn): ResNet(
      (input_stem): ResNetStem(
        (conv_0): Conv_Bn_Activation(
          (conv1d): Conv1d(12, 64, kernel_size=(17,), stride=(1,), padding=(8,))
          (batch_norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation_relu): ReLU()
        )
      )
      (ResNetBottleNeck_0_0): ResNetBottleNeck(
        (shortcut): DownSample(
          (down_sample): Conv1d(64, 512, kernel_size=(1,), stride=(4,), bias=False)
          (batch_normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (main_stream): Sequential(
          (cba_head): Conv_Bn_Activation(
            (conv1d): Conv1d(64, 128, kernel_size=(1,), stride=(1,), bias=False)
            (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activation_relu): ReLU(inplace=True)
          )
          (d

In [10]:
trainer = CINC2020Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

log file path: /home/wenhao/Jupyter/wenhao/workspace/torch_ecg/benchmarks/train_crnn_cinc2020/log/TorchECG_12-12_10-15_ECG_CRNN_CINC2020_adamw_amsgrad_LR_0.0001_BS_64_resnet_nature_comm_bottle_neck_se.txt
level of c_handler is set INFO, level of f_handler is set DEBUG
TorchECG - INFO - training configurations are as follows:
{
    "debug": True,
    "final_model_name": None,
    "log_step": 20,
    "flooding_level": 0.0,
    "early_stopping": {
        "min_delta": 0.001,
        "patience": 10
    },
    "log_dir": "/home/wenhao/Jupyter/wenhao/workspace/torch_ecg/benchmarks/train_crnn_cinc2020/log",
    "checkpoints": "/home/wenhao/Jupyter/wenhao/workspace/torch_ecg/benchmarks/train_crnn_cinc2020/checkpoints",
    "prefix": "TorchECG",
    "torch_dtype": torch.float32,
    "np_dtype": <class 'numpy.float32'>,
    "db_dir": "/home/wenhao/Jupyter/wenhao/data/CinC2021/",
    "model_dir": "/home/wenhao/Jupyter/wenhao/workspace/torch_ecg/benchmarks/train_crnn_cinc2020/checkpoints",
    "ke

In [11]:
len(classes)

24

In [12]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

TorchECG - INFO - 
Starting training:
------------------
Epochs:          50
Batch size:      64
Learning rate:   0.0001
Training size:   35839
Validation size: 9024
Device:          cuda
Optimizer:       adamw_amsgrad
Dataset classes: ['AF', 'AFL', 'Brady', 'IAVB', 'IRBBB', 'LAD', 'LAnFB', 'LBBB', 'LPR', 'LQRSV', 'LQT', 'NSIVCB', 'NSR', 'PAC', 'PR', 'PVC', 'QAb', 'RAD', 'RBBB', 'SA', 'SB', 'STach', 'TAb', 'TInv']
-----------------------------------------



TorchECG - INFO - Train epoch_0:
--------------------------------------------------------------------------------------------------------------
TorchECG - INFO - Train Metrics:
--------------------------------------------------
Epoch 0 / Step 20: train/loss : 0.1559
Epoch 0 / Step 20: train/lr :   0.0001
--------------------------------------------------
TorchECG - INFO - Train Metrics:
--------------------------------------------------
Epoch 0 / Step 40: train/loss : 0.1554
Epoch 0 / Step 40: train/lr :   0.0001
--------------------------------------------------
TorchECG - INFO - Train Metrics:
--------------------------------------------------
Epoch 0 / Step 60: train/loss : 0.1813
Epoch 0 / Step 60: train/lr :   0.0001
--------------------------------------------------
TorchECG - INFO - Train Metrics:
--------------------------------------------------
Epoch 0 / Step 80: train/loss : 0.1432
Epoch 0 / Step 80: train/lr :   0.0001
--------------------------------------------------
Torc

TorchECG - INFO - 
----------------------------------------------
scalar prediction:    [0.045, 0.064, 0.073, 0.193, 0.104, 0.773, 0.436, 0.188, 0.11, 0.119, 0.095, 0.182, 0.958, 0.077, 0.055, 0.097, 0.129, 0.046, 0.026, 0.188, 0.075, 0.025, 0.462, 0.094]
binary prediction:    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels:               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predicted classes:    ['LAD', 'NSR']
label classes:        ['NSR']
----------------------------------------------

- AUROC and AUPRC...
- Accuracy...
- F-measure...
- F-beta and G-beta measures...
- Challenge metric...
Done.
TorchECG - INFO - Train Metrics:
--------------------------------------------------
Epoch 0 / Step 560: train/auroc :            0.8137
Epoch 0 / Step 560: train/auprc :            0.2719
Epoch 0 / Step 560: train/accuracy :         0.3498
Epoch 0 / Step 560: train/f_measure :        0.1836
Epoch 0 / Step 560: train/f_beta_meas

TorchECG - INFO - Train epoch_1:
--------------------------------------------------------------------------------------------------------------
TorchECG - INFO - Train Metrics:
--------------------------------------------------
Epoch 1 / Step 580: train/loss : 0.1142
Epoch 1 / Step 580: train/lr :   0.0001
--------------------------------------------------
TorchECG - INFO - Train Metrics:
--------------------------------------------------
Epoch 1 / Step 600: train/loss : 0.1152
Epoch 1 / Step 600: train/lr :   0.0001
--------------------------------------------------
TorchECG - INFO - Train Metrics:
--------------------------------------------------
Epoch 1 / Step 620: train/loss : 0.1019
Epoch 1 / Step 620: train/lr :   0.0001
--------------------------------------------------
TorchECG - INFO - Train Metrics:
--------------------------------------------------
Epoch 1 / Step 640: train/loss : 0.0969
Epoch 1 / Step 640: train/lr :   0.0001
-----------------------------------------------

TorchECG - INFO - 
----------------------------------------------
scalar prediction:    [0.201, 0.108, 0.03, 0.12, 0.185, 0.166, 0.203, 0.132, 0.049, 0.106, 0.086, 0.109, 0.114, 0.224, 0.118, 0.137, 0.125, 0.136, 0.305, 0.208, 0.182, 0.428, 0.129, 0.167]
binary prediction:    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
labels:               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
predicted classes:    ['STach']
label classes:        ['STach']
----------------------------------------------

- AUROC and AUPRC...
- Accuracy...
- F-measure...
